In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cryptography.fernet import Fernet
import datetime as dt

In [ ]:
dfPrinc = pd.read_csv('../../dados/dfPrincipal.csv')

In [ ]:
dfPrinc.head()

# Analise Dados Alunos

## Análise da % de conclusão(Todos os Alunos)

In [ ]:
sevasoesDF = dfPrinc.groupby(['ANO_EVASAO','FORMA_EVASAO'], dropna=False).size().reset_index(name='ALUNOS')

In [ ]:
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('ABA - Abandono do curso', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('APO - Aluno Especial - Disciplina Isolada', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('APO -Conclusão a Mobilidade Acadêmica Externa - ANDIFES', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('APO -Conclusão da Mobilidade Acadêmica Externa - IAE/IESCE', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('Desistência SiSU', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('JUB - Jubilamento', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('CAN - Cancelamento Geral do curso', 'Evasão') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('CON - Curso concluído', 'Concluído') 
evasoesDF['FORMA_EVASAO'] = evasoesDF.FORMA_EVASAO.str.replace('Sem evasão', 'Cursando') 

In [ ]:
evasoesDF.loc[evasoesDF['FORMA_EVASAO'] == 'Não identificada (vide pasta do aluno)' , 'FORMA_EVASAO'] = 'Evasão'

In [ ]:
evasoesDF = evasoesDF.groupby(['FORMA_EVASAO'] , as_index=False)['ALUNOS'].sum() 

In [ ]:
# Mostra porcentagem de alunos
evasoesDF['PERCENT'] =  (100. * evasoesDF['ALUNOS'] / evasoesDF['ALUNOS'].sum()).round(0)


In [ ]:
evasoesDF.to_csv ('../../dados/evasoesDF.csv', index = False, header=True)
print(evasoesDF)

## Análise da % de gênero

In [ ]:
generoDF = dfPrinc.groupby(['SEXO'], dropna=False).size().reset_index(name='ALUNOS')

In [ ]:
generoDF['PERCENT'] =  (100. * generoDF['ALUNOS'] / generoDF['ALUNOS'].sum()).round(0)

In [ ]:
generoDF.loc[(generoDF['SEXO'] == 'M'),'SEXO']='Masculino'
generoDF.loc[(generoDF['SEXO'] == 'F'),'SEXO']='Feminino'

In [ ]:
generoDF.to_csv ('../../dados/generoDF.csv', index = False, header=True)
print(generoDF)

## Análise do perfil de idade


In [ ]:
dfPrinc['DT_NASCIMENTO'] = pd.to_datetime(dfPrinc['DT_NASCIMENTO'])


In [ ]:
#colunas = ['SEXO', 'FORMA_INGRESSO','FORMA_EVASAO','PERIODO_INGRESSO','DT_EVASAO','PERIODO_EVASAO','PER_EVASAO_FORMAT','ANO_EVASAO',
#           'PER_INGRESSO_FORMAT']
colunas = ['DT_NASCIMENTO','ANO_INGRESSO']
idadeDF = dfPrinc.filter(colunas)


In [ ]:
arrayDatas = dfPrinc['DT_NASCIMENTO'].array
arrayIngresso = dfPrinc['ANO_INGRESSO'].array
arrayIdadeFormatado = []
def getIdadeIngresso(DT_NASCIMENTO, ANO_INGRESSO):
    return arrayIdadeFormatado.append(ANO_INGRESSO - DT_NASCIMENTO.year)
    

In [ ]:
for dt,ing in zip(arrayDatas,arrayIngresso):
    getIdadeIngresso(dt, ing)

In [ ]:
idadeDF['IDADE'] = arrayIdadeFormatado
idadeDF = idadeDF.fillna(0)
idadeDF['IDADE'] = idadeDF['IDADE'].astype(int)

In [ ]:
idadeDF = idadeDF.groupby(['IDADE'], dropna=True).size().reset_index(name='ALUNOS')

In [ ]:
idadeDF = idadeDF.drop(0)

In [ ]:
idadeDF['PERCENT'] =  (100. * idadeDF['ALUNOS'] / idadeDF['ALUNOS'].sum()).round(1)

In [ ]:
idadeDF.set_index('IDADE', inplace=True)

In [ ]:
nidadeDF = idadeDF.T

In [ ]:
idadeDF
nidadeDF['17-21'] = nidadeDF.iloc[:,0:4].sum(axis=1)
#df.loc[len(df.index)] = ['Amy', 89, 93]

#nidadeDF = idadeDF.T 
nidadeDF['22-26'] = nidadeDF.iloc[:,5:9].sum(axis=1)
nidadeDF['27-31'] = nidadeDF.iloc[:,10:14].sum(axis=1)
nidadeDF['32-36'] = nidadeDF.iloc[:,15:19].sum(axis=1)
nidadeDF['37-41'] = nidadeDF.iloc[:,20:24].sum(axis=1)
nidadeDF['< 41 > 51'] = nidadeDF.iloc[:,25:34].sum(axis=1)
nidadeDF['< 51'] = nidadeDF.iloc[:,35:42].sum(axis=1)


In [ ]:
nidadeDF = nidadeDF.drop(nidadeDF.iloc[:, 0:42],axis = 1)

In [ ]:
nidadeDF = nidadeDF.T

In [ ]:
nidadeDF.to_csv ('../../dados/idadeDF.csv', index = True, header=True)
print(idadeDF)

# Análise da forma de ingresso

In [ ]:
forma_ingresso_mapping = {
    'AE - Aluno Especial - Disciplina Isolada': 'Outros',
    'AE - Mobilidade Acadêmica IAE/IESCE': 'Outros',
    'AE - Mobilidade Adadêmica Externa - ANDIFES': 'Outros',
    'DJ - Decisão Judicial': 'Outros',
    'EN - ENEM': 'Ampla Concorrência',
    'EO - Transferência Ex-Ofício': 'Outros',
    'PD - Portador de Diploma de Nível Superior': 'Outros',
    'Pessoas com Deficiência': 'Outros',
    'PG - Programa de Estudantes - Convênio (PEC-G)': 'Outros',
    'SISU Ampla Concorrência': 'Ampla Concorrência',
    'SISU EscolaPública - Indep. de Renda': 'Cotas',
    'SISU Escola Pública até 1,5 S.M Índio': 'Cotas',
    'SISU Escola Pública até 1,5 S.M Preto e Pardo': 'Cotas',
    'SISU Escola Pública até 1,5 S.M.': 'Cotas',
    'SISU Escola Pública até 1,5 S.M. Preto, Pardo, Indígena': 'Cotas',
    'SISU Escola Pública, Indep. de Renda: Preto, Pardo, Indígena': 'Cotas',
    'SISU Escola Pública, Indep. de Renda: Índio': 'Cotas',
    'SISU Escola Pública, Indep. de Renda: Preto e Pardo': 'Cotas',
    'TE - Transferência Externa - oriunda de outra instituição': 'Outros',
    'TC - Transferência Interna-Curso não relacionado ao anterior': 'Outros',
    'VE - Vestibular': 'Ampla Concorrência'
}

# Substituir as categorias
dfPrinc['FORMA_INGRESSO'] = dfPrinc['FORMA_INGRESSO'].replace(forma_ingresso_mapping)